In [2]:
from BigDFT import Datasets as D, Calculators as C, Inputfiles as I, Logfiles as lf
from BigDFT.Database import Molecules
from futile.Utils import write
import numpy as np
import matplotlib.pyplot as plt
import os, sys, yaml
sys.path.insert(0,'../')
import StatPol as SP, Routines as R

basepath = os.getcwd()

## Analysis of the statical polarizability for the nsp dataset

We read the nsp_dataset from file. Note that the file nsp_dataset.yaml contains only the reference results and the information on the allowed studies. So, to update the actual set of results with new computations we have to load
the nsp_results.yaml. In this way the new computations performed will increase the dictionary. 

In [3]:
nsp_dataset = yaml.load(open('nsp_dataset.yaml'))
#nsp_dataset = yaml.load(open('nsp_results.yaml'))
#nsp_dataset

For testing and computation purposes we split the dataset into several subset made of
10 molecules each (molecules are sorted in alphabetical order before the splitting). So we have
8 subsets (the last one with 5 molecules)

In [4]:
molecules = nsp_dataset.keys()
molecules.sort()
print 'number of nsp elements ', len(molecules)
subset_length = 10

subset = {}
for ind in range(int(len(molecules)/subset_length)+1):
    subset[ind] = []

for ind,mol in enumerate(molecules):
    subset[int(ind/subset_length)].append(mol)

number of nsp elements  75


In [5]:
computed_mol = []
for mol,data in nsp_dataset.iteritems():
    if 'results' in data:
        computed_mol.append(mol)
computed_mol.sort()
print len(computed_mol)
#computed_mol

0


In [7]:
subset[0]

['AlF', 'Ar', 'BF', 'BH2Cl', 'BH2F', 'BH3', 'BHF2', 'BeH2', 'C2H2', 'C2H4']

We also define a reduced set of studies (pbe0 is not computed in this moment)

In [9]:
reduced_study_set = [('lda_pt','hgh_k'),('lda_pw','hgh_k'),('pbe','hgh_k'),\
                     ('pbe','nlcc_aw'),('pbe','nlcc_ss')] #('pbe0','hgh_k')

Run the nsp_worflow for the nsp_dataset. For each molecule we add the key 'results'. This key contain the results, for all the allowed studies. 

In [10]:
reload(R)
code=C.SystemCalculator(omp=2,mpi_run='mpirun -np 4',skip=True)

for mol in ['CO']: #subset[0]:
    data = nsp_dataset[mol]
    data['results'] = {}
    for study in data['study']:
        if study in reduced_study_set:
            data['results'][study] = {}
            data['results'][study] = R.nsp_workflow(term_verb=True,molecule=mol,study=study,code=code)
            print ''

/home/marco/Data/RICERCA/DFT AND MANY BODY/LR MOLECULES/Statical polarizability/Analysis/Calculations
Initialize a Calculator with OMP_NUM_THREADS=2 and command mpirun -np 4 /home/marco/Applications/BigDFT/binaries/v1.8.3/install/bin/bigdft
Compute alpha for :  CO lda_pt hgh_k
Seek for gs convergence
Perform the run with rmult 3.0
Creating the yaml input file "lda_pt-hgh_k/rmult:3.0.yaml"
Run directory lda_pt-hgh_k
Executing command:  mpirun -np 4 /home/marco/Applications/BigDFT/binaries/v1.8.3/install/bin/bigdft -n rmult:3.0 -s Yes
Perform the run with rmult 4.0
Creating the yaml input file "lda_pt-hgh_k/rmult:4.0.yaml"
Run directory lda_pt-hgh_k
Executing command:  mpirun -np 4 /home/marco/Applications/BigDFT/binaries/v1.8.3/install/bin/bigdft -n rmult:4.0 -s Yes
Convergence for rmult 3.0 failed
Perform the run with rmult 5.0
Creating the yaml input file "lda_pt-hgh_k/rmult:5.0.yaml"
Run directory lda_pt-hgh_k
Executing command:  mpirun -np 4 /home/marco/Applications/BigDFT/binaries/

In [17]:
r = nsp_dataset['CO']['results']['lda_pt','hgh_k']['rmult_conv']['converged_value']
SP.eval_alpha_avg(nsp_dataset['CO']['results']['lda_pt','hgh_k']['rmult_conv']['results'][r])

13.710287999999998

In [18]:
r = nsp_dataset['CO']['results']['lda_pw','hgh_k']['rmult_conv']['converged_value']
SP.eval_alpha_avg(nsp_dataset['CO']['results']['lda_pw','hgh_k']['rmult_conv']['results'][r])

40.577220000000004

Save the dataset as yaml file

In [10]:
import yaml
with open('nsp_results.yaml', 'w') as outfile:
    yaml.dump(nsp_dataset, outfile, default_flow_style=False)

## Analysis of the statical polarizability for the sp dataset

....TO BE DONE....